In [ ]:
!pip3 install --upgrade  langchain langchain-community langchain-openai chromadb 
!pip3 install --upgrade  pypdf pandas streamlit python-dotenv

^C


In [5]:
# Import Langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

# Other modules and packages
import os
import tempfile
import streamlit as st  
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

## Define our LLM

In [6]:


llm = ChatGroq(
    temperature=0,
    timeout=None,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

## Process PDF document

### Load PDF document

In [13]:
loader = PyPDFLoader("cv.pdf")
pages = loader.load()


for page in pages:
    print(page)

page_content='anounman.app |015751635689 | ankushdas67824@gmail.c om
EDUCATION
UNIVERSITÄT DES
SAARLANDES
B-tech in Computer Science and
Technology
Ba tch o f 2024 |
S aarbucken,Deu tschland
A.C.INSTITUTION
class XII ba tch o f 2022 | Mal da, India
class X ba tch o f 2020 | Mal da, India
LINKS
Github:// anounman
LinkedIn:// anounman
Twitter:// anounman
LANGUAGE
Deutsch
Full Professional Proficiency
English
Full Professional Proficiency
Hindi
Native orBilingual Proficiency
INTERESTS
Friction andnon-friction Books
Cryptocurrency
Cinematography
SKILLS
PROGRAMMING
Language:
Python •Dart •SQL •C•C++
Technologies:
Flutter •HTML •CSS •React.js
•Node.js •Express.js •Mongo.Db
•AWS EC2 •Linux •Firebase
Familiar:
Github •Figma •OpenCV
•Django• Fast Api
CERTIFICATES
Website Hacking /Penetration T esting
and Bug Bounty Hunting.
Learn Python and Ethical Hacking From
Scratch.
District Student-Youth Science Fair 2017.
District Student-Youth Science Fair 2018EXPERIENCE
QUICK RESERVED GMBH | Flutter Dev

### Split document

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)


for chunk in chunks:
    print(chunk)

page_content='anounman.app |015751635689 | ankushdas67824@gmail.c om
EDUCATION
UNIVERSITÄT DES
SAARLANDES
B-tech in Computer Science and
Technology
Ba tch o f 2024 |
S aarbucken,Deu tschland
A.C.INSTITUTION
class XII ba tch o f 2022 | Mal da, India
class X ba tch o f 2020 | Mal da, India
LINKS
Github:// anounman
LinkedIn:// anounman
Twitter:// anounman
LANGUAGE
Deutsch
Full Professional Proficiency
English
Full Professional Proficiency
Hindi
Native orBilingual Proficiency
INTERESTS
Friction andnon-friction Books
Cryptocurrency
Cinematography
SKILLS
PROGRAMMING
Language:
Python •Dart •SQL •C•C++
Technologies:
Flutter •HTML •CSS •React.js
•Node.js •Express.js •Mongo.Db
•AWS EC2 •Linux •Firebase
Familiar:
Github •Figma •OpenCV
•Django• Fast Api
CERTIFICATES
Website Hacking /Penetration T esting
and Bug Bounty Hunting.
Learn Python and Ethical Hacking From
Scratch.
District Student-Youth Science Fair 2017.
District Student-Youth Science Fair 2018EXPERIENCE
QUICK RESERVED GMBH | Flutter Dev

### Create embeddings

In [16]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()
test_vector = embedding_function.embed_query("cat")

NameError: name 'OPENAI_API_KEY' is not defined

In [10]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance", 
                            embeddings=embedding_function)

evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

{'score': 0.17440875566198188}

In [13]:
evaluator.evaluate_strings(prediction="Paris", reference="coffeeshop")

{'score': 0.22417909850229667}

### Create vector database

In [18]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [19]:
# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks, 
                                 embedding_function=embedding_function, 
                                 vectorstore_path="vectorstore_test")

## 2. Query for relevant data

In [59]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

In [21]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the title of the paper?")
relevant_chunks

[Document(metadata={'page': 1, 'source': 'data/1995-watanabe.pdf'}, page_content='Am. J. Physiol. 250, G85~91. \nWatanabe S. and Dawes C. (1988) The effects of different \nfoods and concentrations of citric acid on the flow rate of \nwhole saliva in man. Archs oral Biol. 33, 1-5. \nWatanabe S. and Dawes C. (1990) Salivary flow rates and \nsalivary film thickness in five-year-old children. J. dent. \nRes. 69, 1150-1153. \nWatanabe S. (1992) Salivary clearance from different re- \ngions of the mouth in children. Caries Res. 26, 423-427.'),
 Document(metadata={'page': 1, 'source': 'data/1995-watanabe.pdf'}, page_content='grants-in-aid from the Ministry of Education, Science, and \nCulture of Japan, Grants 02807189, 05671719 (to S.W.), and \nby a Grant from the Research Foundation of the Health \nSciences University of Hokkaido (to S.W.). \nREFERENCES \nBecks H. and Wainwright W. W. (1943) Human saliva XIII. \nRate of flow of resting saliva of healthy individuals. \nJ. dent. Res. 22, 391-3

In [ ]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

## 3. Generate responses

In [62]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the title of the paper?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

Copyright#2005 John Wiley & Sons, Ltd. Appl. Cognit. Psychol. 20: 139–156 (2006)

---

language?’ 86.4% of the sample admitted to having done so. Nearly two-thirds answered yes to the question, ‘When you write an essay, do you turn to the thesaurusto choose words that are more complex to give the impression that the content is morevalid or intelligent?’Copyright#2005 John Wiley & Sons, Ltd.*Correspondence to: D. M. Oppenheimer, Department of Psychology, Princeton University, Green Hall Room2-S-8, Princeton, NJ 08540, USA. E-mail: doppenhe@princeton.edu

---

APPLIED COGNITIVE PSYCHOLOGYAppl. Cognit. Psychol.20: 139–156 (2006)Published online 31 October 2005 in Wiley InterScience(www.interscience.wiley.com) DOI: 10.1002/acp.1178Consequences of Erudite Vernacular Utilized Irrespecti

In [65]:
llm.invoke(prompt)

AIMessage(content='The title of the paper is "Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 773, 'total_tokens': 805}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2017e06-5717-4a90-a7dc-47634ed2728f-0', usage_metadata={'input_tokens': 773, 'output_tokens': 32, 'total_tokens': 805})

### Using Langchain Expression Language

In [66]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

AIMessage(content='The title of the paper is "Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 767, 'total_tokens': 799}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-b478d845-c13f-4ee4-868d-a199f8534a9f-0', usage_metadata={'input_tokens': 767, 'output_tokens': 32, 'total_tokens': 799})

### Generate structured responses

In [73]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")
    
class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [74]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo, strict=True)
        )

rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")

ExtractedInfo(paper_title=AnswerWithSources(answer='Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly', sources='Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly', reasoning='The title is explicitly mentioned in the context provided.'), paper_summary=AnswerWithSources(answer='Most texts on writing style encourage authors to avoid overly-complex words. However, a majority of undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give the impression of intelligence. This paper explores the extent to which this strategy is effective. Experiments 1–3 manipulate complexity of texts and find a negative relationship between complexity and judged intelligence.', sources='Most texts on writing style encourage authors to avoid overly-complex words. However, a majority of undergraduates admit to deliberately increasing the complexity o

### Transform response into a dataframe

In [75]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response.dict()])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

,paper_title,paper_summary,publication_year,paper_authors
answer,Consequences of Erudite Vernacular Utilized Ir...,The paper explores the negative relationship b...,2006,Daniel M. Oppenheimer
source,"Copyright#2005 John Wiley & Sons, Ltd. Appl. C...",Most texts on writing style encourage authors ...,Appl. Cognit. Psychol. 20: 139–156 (2006),"Correspondence to: D. M. Oppenheimer, Departme..."
reasoning,The title is explicitly mentioned at the begin...,The summary is derived from the overall conten...,The publication year is indicated in the citat...,The author’s name is provided in the correspon...
